In [1]:
import librosa
import argparse
import json
import os
import time
import shutil
import pandas as pd
import numpy as np
import pickle as pkl 
from PIL import Image
import lmdb
import random

import torch
import torch.nn as nn
from torch.utils.data import *
import torchaudio
import torchvision
import torchvision.models as models
from torchsummary import summary

from tqdm import tqdm
from tensorboardX import SummaryWriter

import wandb

In [2]:
wandb.login()

wandb: Currently logged in as: charleestefanski (chars). Use `wandb login --relogin` to force relogin


True

In [3]:
%cd

/home/ubuntu


In [4]:
class MelSpectrogram(object):
    def __init__(self, bins, mode, dataset):
        self.window_length = [25, 50, 100]
        self.hop_length = [10, 25, 50]
        self.fft = 4410
        self.melbins = bins
        self.mode = mode
        self.sr = 44100
        self.length = 250
    def __call__(self, value):
        sample = value
        limits = ((-2, 2), (0.9, 1.2))

        if self.mode=="train":
            pitch_shift = np.random.randint(limits[0][0], limits[0][1] + 1)
            time_stretch = np.random.random() * (limits[1][1] - limits[1][0]) + limits[1][0]
            new_audio = librosa.effects.time_stretch(y = librosa.effects.pitch_shift(y = sample, sr = self.sr, n_steps = pitch_shift), rate = time_stretch)
        else:
            pitch_shift = 0
            time_stretch = 1
            new_audio = sample
        specs = []
        for i in range(len(self.window_length)):
            clip = torch.Tensor(new_audio)

            window_length = int(round(self.window_length[i]*self.sr/1000))
            hop_length = int(round(self.hop_length[i]*self.sr/1000))
            spec = torchaudio.transforms.MelSpectrogram(sample_rate=self.sr, n_fft=self.fft, win_length=window_length, hop_length=hop_length, n_mels=self.melbins)(clip)
            eps = 1e-6
            spec = spec.numpy()
            spec = np.log(spec+ eps)
            spec = np.asarray(torchvision.transforms.Resize((128, self.length))(Image.fromarray(spec)))
            specs.append(spec)
        specs = np.array(specs).reshape(-1, 128, self.length)
        specs = torch.Tensor(specs)
        return specs

class AudioDataset(Dataset):
    def __init__(self, pkl_dir, dataset_name, transforms=None):
        self.transforms = transforms
        self.data = []
        self.length = 1500 if dataset_name=="GTZAN" else 250
        with open(pkl_dir, "rb") as f:
            self.data = pkl.load(f)
    def __len__(self):
        if self.transforms.mode == "train":
            return 2*len(self.data)
        else:
            return len(self.data)
    def __getitem__(self, idx):
        if idx >= len(self.data):
            new_idx = idx - len(self.data)
            entry = self.data[new_idx]
            if self.transforms:
                values = self.transforms(entry["audio"])
        else:
            entry = self.data[idx]
            values = torch.Tensor(entry["values"].reshape(-1, 128, self.length))
        target = torch.LongTensor([entry["target"]])
        return (values, target)

def fetch_dataloader(pkl_dir, dataset_name, batch_size, num_workers, mode):
    transforms = MelSpectrogram(128, mode, dataset_name)
    dataset = AudioDataset(pkl_dir, dataset_name, transforms=transforms)
    dataloader = DataLoader(dataset,shuffle=True, batch_size=batch_size, num_workers=num_workers)
    return dataloader

In [5]:
class ResNet(nn.Module):
    def __init__(self, dataset, pretrained=True):
        super(ResNet, self).__init__()
        num_classes = 50 if dataset=="ESC" else 10
        self.model = models.resnet50(pretrained=pretrained)
        self.model.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        output = self.model(x)
        return output

In [6]:
# parameters
dataset_name = "ESC"
data_dir = "DataSci251_FinalProject/DataSet/ESC-50-master/store_spectograms"
dataaug = True
pretrained = True
scheduler = True,
model = "resnet"
batch_size = 32
num_workers = 8
epochs = 70
lr = 1e-4
weight_decay = 1e-3
num_folds = 5
checkpoint_dir = "DataSci251_FinalProject/DataSet/ESC-50-master/checkpoint_dir_resnet"

In [15]:
run = wandb.init(
    # Set the project where this run will be logged
    project="audio_densenet",
    # Track hyperparameters and run metadata
    config={
        "model": 'resnet',
        "batch_size":batch_size,
        "learning_rate": lr,
        "epochs": epochs,
    })

accuracy,▁
loss,▁
accuracy,77.0
loss,2.1174


In [8]:
class RunningAverage():
    def __init__(self):
        self.total = 0
        self.steps = 0
    def update(self, loss):
        self.total += loss
        self.steps += 1
    def __call__(self):
        return (self.total/float(self.steps))

In [9]:
def save_checkpoint(state, is_best, split, checkpoint):
    filename = os.path.join(checkpoint, 'last{}.pth.tar'.format(split))
    if not os.path.exists(checkpoint):
        print("Checkpoint Directory does not exist")
        os.mkdir(checkpoint)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(checkpoint, "model_best_{}.pth.tar".format(split)))

In [10]:
def train(model, device, data_loader, optimizer, loss_fn):
    model.train()
    loss_avg = RunningAverage()

    with tqdm(total=len(data_loader)) as t:
        for batch_idx, data in enumerate(data_loader):
            inputs = data[0].to(device)
            target = data[1].squeeze(1).to(device)

            outputs = model(inputs)

            loss = loss_fn(outputs, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_avg.update(loss.item())

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()
    return loss_avg()

In [11]:
def evaluate(model, device, test_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            inputs = data[0].to(device)
            target = data[1].squeeze(1).to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return (100*correct/total)

In [12]:
def train_and_evaluate(model, device, train_loader, val_loader, optimizer, loss_fn, writer, epochs, checkpoint_dir, split, scheduler=None):
    best_acc = 0.0

    for epoch in range(epochs):
        avg_loss = train(model, device, train_loader, optimizer, loss_fn)

        acc = evaluate(model, device, val_loader)
        print("Epoch {}/{} Loss:{} Valid Acc:{}".format(epoch, epochs, avg_loss, acc))
        
        wandb.log({"accuracy": acc, "loss": avg_loss})

        is_best = (acc > best_acc)
        if scheduler:
            scheduler.step()
        if is_best:
            best_acc = acc
            filename = os.path.join("{}".format(checkpoint_dir), 'myModel_export.pt')
            model_cpu = model.to('cpu')
            model_scripted = torch.jit.script(model_cpu)
            model_scripted.save(filename)
            
            model = model.to('cuda')

        save_checkpoint({"epoch": epoch + 1,
                               "model": model.state_dict(),
                               "optimizer": optimizer.state_dict()}, is_best, split, "{}".format(checkpoint_dir))
        writer.add_scalar("data{}/trainingLoss{}".format(dataset_name, split), avg_loss, epoch)
        writer.add_scalar("data{}/valLoss{}".format(dataset_name, split), acc, epoch)
    writer.close()

In [16]:
## need config path
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for i in range(1, num_folds+1):
    if dataaug:
        train_loader = fetch_dataloader( "{}training128mel{}.pkl".format(data_dir, i), "ESC", batch_size, num_workers, 'train')
        val_loader = fetch_dataloader("{}validation128mel{}.pkl".format(data_dir, i), "ESC", batch_size, num_workers, 'validation')
    else:
        print("something wrong")

    writer = SummaryWriter(comment="ESC")
    model = ResNet("ESC", pretrained).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    if scheduler:
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, gamma=0.1)
    else:
        scheduler = None

    train_and_evaluate(model, device, train_loader, val_loader, optimizer, loss_fn, writer,epochs, checkpoint_dir, i, scheduler)

100%|█████████████████████████████| 100/100 [03:34<00:00,  2.15s/it, loss=2.118]


Epoch 0/70 Loss:2.118273896574974 Valid Acc:74.5


100%|█████████████████████████████| 100/100 [03:35<00:00,  2.15s/it, loss=0.459]


Epoch 1/70 Loss:0.4585291515290737 Valid Acc:79.75


100%|█████████████████████████████| 100/100 [03:35<00:00,  2.16s/it, loss=0.169]


...

Epoch 43/70 Loss:0.0050342719780746845 Valid Acc:89.0


100%|█████████████████████████████| 100/100 [03:36<00:00,  2.17s/it, loss=0.004]


Epoch 44/70 Loss:0.004260176977841184 Valid Acc:89.0


  0%|                                                   | 0/100 [00:02<?, ?it/s]


KeyboardInterrupt: 